In [3]:
#Dependencies
import tweepy
import json
import numpy as np

from config import consumer_key, consumer_secret, access_token, access_token_secret

In [4]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [5]:
# Twitter API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret

In [6]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [7]:
target_terms = ("@BBC", "@CBSNews", "@CNN", "@FoxNews", "@nytimes")
# Loop through all target users
for target in target_terms:

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times (total of 1500 tweets)
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(target, count=100, result_type="recent")

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:
                # Run Vader Analysis on each tweet
                compound = analyzer.polarity_scores(tweet["text"])["compound"]
                pos = analyzer.polarity_scores(tweet["text"])["pos"]
                neu = analyzer.polarity_scores(tweet["text"])["neu"]
                neg = analyzer.polarity_scores(tweet["text"])["neg"]

                # Add each value to the appropriate array
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)



In [11]:
positive_list

[0.114,
 0.181,
 0.0,
 0.114,
 0.183,
 0.106,
 0.183,
 0.196,
 0.114,
 0.196,
 0.0,
 0.16,
 0.0,
 0.738,
 0.114,
 0.0,
 0.16,
 0.0,
 0.114,
 0.0,
 0.183,
 0.106,
 0.114,
 0.0,
 0.343,
 0.114,
 0.0,
 0.0,
 0.0,
 0.273,
 0.114,
 0.0,
 0.185,
 0.0,
 0.122,
 0.16,
 0.114,
 0.114,
 0.25,
 0.0,
 0.0,
 0.16,
 0.114,
 0.183,
 0.0,
 0.0,
 0.0,
 0.0,
 0.114,
 0.0,
 0.0,
 0.114,
 0.385,
 0.0,
 0.0,
 0.16,
 0.183,
 0.196,
 0.0,
 0.0,
 0.139,
 0.0,
 0.114,
 0.114,
 0.0,
 0.26,
 0.114,
 0.114,
 0.0,
 0.106,
 0.183,
 0.183,
 0.434,
 0.114,
 0.0,
 0.0,
 0.114,
 0.0,
 0.343,
 0.114,
 0.0,
 0.0,
 0.0,
 0.106,
 0.0,
 0.0,
 0.0,
 0.16,
 0.106,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.16,
 0.0,
 0.16,
 0.139,
 0.132,
 0.196,
 0.114,
 0.181,
 0.0,
 0.114,
 0.183,
 0.106,
 0.183,
 0.196,
 0.114,
 0.196,
 0.0,
 0.16,
 0.0,
 0.738,
 0.114,
 0.0,
 0.16,
 0.0,
 0.114,
 0.0,
 0.183,
 0.106,
 0.114,
 0.0,
 0.343,
 0.114,
 0.0,
 0.0,
 0.0,
 0.273,
 0.114,
 0.0,
 0.185,
 0.0,
 0.122,
 0.16,
 0.114,
 0.114,
 0.25